In [1]:
import fasttext
import pandas as pd
import os
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import re
import csv
import numpy as np

### creating pipeline for preprocessing texts

In [ ]:
catalog.load("s")

In [36]:
train = pd.read_csv("data/01_raw/training.txt", sep='\t', header=0, encoding="ISO-8859-1")

In [20]:
test = pd.read_csv("data/01_raw/test.txt", sep='\t', header=0, encoding="ISO-8859-1")

In [28]:
train.to_csv("data/01_raw/training.csv",index=False)

In [29]:
test.to_csv("data/01_raw/test.csv",index=False)

In [39]:
train = pd.read_csv("data/01_raw/training.csv")

In [40]:
test = pd.read_csv("data/01_raw/test.csv")

### creating cooking reader

In [16]:
data = pd.read_csv("../training data/cooking/cooking.train", sep="|", header=None, names=["line"])

In [17]:
data["line replaced"] = data["line"].apply(lambda x: x.replace(" __label__", "__label__"))

In [18]:
data["label list spliited"] = data["line replaced"].apply(lambda x: x.split(" ",1))

In [19]:
data["text"] =  data["label list spliited"].apply(lambda x: x[1])

In [20]:
data["label list"] = data["label list spliited"].apply(lambda x: x[0])

In [21]:
data_splitted = data[["label list","text"]]

In [22]:
data_splitted["label list"] = data_splitted["label list"].apply(lambda x:x.split("__label__")[1:])

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [131]:
final_data = pd.DataFrame(data_splitted["label list"].values.tolist(), index= data_splitted.index, columns = ["__label__"+str(i) for i in range(1,6)])
final_data["text"] = data_splitted["text"]
final_data

,__label__1,__label__2,__label__3,__label__4,__label__5,text
0,sauce,cheese,None,None,None,How much does potato starch affect a cheese sa...
1,food-safety,acidity,None,None,None,Dangerous pathogens capable of growing in acid...
2,cast-iron,stove,None,None,None,How do I cover up the white spots on my cast i...
3,restaurant,None,None,None,None,Michelin Three Star Restaurant; but if the che...
4,knife-skills,dicing,None,None,None,"Without knife skills, how can I quickly and ac..."
5,storage-method,equipment,bread,None,None,What's the purpose of a bread box?
6,baking,food-safety,substitutions,peanuts,None,how to seperate peanut oil from roasted peanut...
7,chocolate,None,None,None,None,American equivalent for British chocolate terms
8,baking,oven,convection,None,None,Fan bake vs bake
9,sauce,storage-lifetime,acidity,mayonnaise,None,Regulation and balancing of readymade packed m...


In [44]:
model = fasttext.train_supervised(input="../training data/cooking.preprocessed.txt")

In [10]:
model.save_model("../models/model_cooking.bin")

In [42]:
model.predict("Which baking dish is best to bake a banana bread ?")

(('__label__bread',), array([0.38804641]))

In [12]:
model.predict("Why not put knives in the dishwasher?")

(('__label__baking',), array([0.06785078]))

In [14]:
model.test("../training data/cooking.valid")

(3000, 0.13066666666666665, 0.056508577194752774)

In [16]:
model.test("../training data/cooking.valid", k=5)

(3000, 0.06633333333333333, 0.1434337609917832)

In [17]:
model.predict("Why not put knives in the dishwasher?", k=5)

(('__label__baking',
  '__label__food-safety',
  '__label__equipment',
  '__label__bread',
  '__label__substitutions'),
 array([0.06785078, 0.06423847, 0.03607798, 0.03567965, 0.03392866]))

In [2]:
def simplify(word):
    dump = ''
    temp = []
    listOfWords = list(filter(None,re.split("([A-Z][^A-Z]*)",word)))
    if len(listOfWords) == len(word):
        return word.lower()
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].lower()
        if len(listOfWords[i]) == 1:
            dump = dump + listOfWords[i]
            if dump in words.words() and len(dump) > 2:
                temp.append(dump)
                dump = ''
        else:
            temp.append(listOfWords[i])
    return temp

In [3]:
stemmer = SnowballStemmer("english")
def createTokens(example_sentence):
    listOfTweets = []
    listOfStances = []
    # Create a sentence using target and the tweet. Word vector will be formed from this.
    # Remove punctuation
    final_sentence = example_sentence.translate(string.punctuation)
    wordList = word_tokenize(final_sentence)
    simpleWordList = []
    s = ' '.join([i for i in wordList if i.isalpha()])
    # create tokens from the string and stem them
    wordList = word_tokenize(s)
    for word in wordList:
        #to break any combined word into its components for eg, hashtags
        simpleWordList += simplify(word)
    # we stem the words to reduce the number of unique words by reducing the words to their origins
    final_sentence = ' '.join(stemmer.stem(word) for word in simpleWordList)
    # All tweets from training data
    
    return final_sentence

In [27]:
createTokens("How much does potato starch affect a cheese sauce recipe?")

'how much doe potato starch affect a chees sauc recip'

In [107]:
model = fasttext.train_supervised(input="../training data/cooking.train")

In [109]:
model.test("../training data/cooking.valid")

(3000, 0.13266666666666665, 0.05737350439671328)

In [112]:
model = fasttext.train_supervised(input="../training data/cooking.train", lr=1.0, epoch=25, wordNgrams=2)

In [114]:
model.test("../training data/cooking.valid")

(3000, 0.5696666666666667, 0.24636009802508288)

In [115]:
model = fasttext.train_supervised(input="../training data/cooking.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [116]:
model.test("../training data/cooking.valid")

(3000, 0.5406666666666666, 0.2338186535966556)

In [118]:
model = fasttext.train_supervised(input="../training data/cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')

In [119]:
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

(('__label__baking', '__label__bread', '__label__bananas'),
 array([1.00001001, 0.99850881, 0.93246335]))

### Making own dataset for stance classification

In [10]:
trainTweets = pd.read_csv("data/training data/stance_classification/stance.train.preprocessed.tokenized.txt", sep='\t',header=0, encoding = "ISO-8859-1")

In [11]:
testTweets = pd.read_csv("data/training data/stance_classification/stance.test.preprocessed.tokenized.txt", sep='\t',header=0, encoding = "ISO-8859-1")

In [12]:
trainTweets["Target"] = trainTweets["Target"].apply(lambda x:x.replace(" ","_").lower())
testTweets["Target"] = testTweets["Target"].apply(lambda x:x.replace(" ","_").lower())

KeyError: 'Target'

#### take all rows where stance is NONE and assign them to one dataframe

In [7]:
trainTweetsWithStance = trainTweets[trainTweets["Stance"] != "NONE"]

In [8]:
trainTweetsWithStanceRelCols = trainTweetsWithStance[["Target","Stance","Tweet"]]

In [9]:
trainTweetsWithStanceRelCols["label"] = "__label__" + trainTweetsWithStanceRelCols["Target"] 

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
#trainTweetsWithStanceRelCols["TweetTokenized"] = trainTweetsWithStanceRelCols["Tweet"].apply(createTokens)

In [11]:
trainTweetsWithStanceRelCols["Final"] = trainTweetsWithStanceRelCols["label"] + " " + trainTweetsWithStanceRelCols["Tweet"]

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
trainTweetsWithStanceRelCols[["Final"]].to_csv("../training data/stance.train.preprocessed.txt",
                                                                 header = None, index=False, sep="|",
                                                                quoting = csv.QUOTE_NONE,
                                                               quotechar = '',escapechar='\\')

In [13]:
trainTweetsWithStanceRelCols.to_csv("stance.dataframe.preprocessed.txt",
                                         header= None)

#### test set

In [118]:
testTweetsWithStance = testTweets[(testTweets["Stance"] != "NONE")]

In [119]:
testTweetsWithStanceRelCols = testTweetsWithStance[["Target","Stance","Tweet"]]

In [120]:
testTweetsWithStanceRelCols["label"] = "__label__"+testTweetsWithStance["Target"]

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
#testTweetsWithStanceRelCols["TweetTokenized"] = testTweetsWithStanceRelCols["Tweet"].apply(createTokens)

In [122]:
testTweetsWithStanceRelCols["Final"] = testTweetsWithStanceRelCols["label"] + " " + testTweetsWithStanceRelCols["Tweet"]

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [123]:
testTweetsWithStanceRelCols[["Final"]].to_csv("../training data/stance.valid.preprocessed.txt",
                                                                 header = None, index=False, sep="|",
                                                                quoting = csv.QUOTE_NONE,
                                                               quotechar = '',escapechar='\\')

In [124]:
testTweetsWithStanceRelCols.to_csv("stance.dataframe.valid.preprocessed.txt",
                                         header= None)

### Build model for text classification

In [ ]:
context.

In [135]:
model = fasttext.train_supervised(input="../training data/stance.train.preprocessed.txt",
                                 lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [136]:
model.get_labels()

['__label__feminist_movement',
 '__label__hillary_clinton',
 '__label__legalization_of_abortion',
 '__label__atheism',
 '__label__climate_change_is_a_real_concern']

In [137]:
model.save_model("../models/model_stances.bin")

In [138]:
model.test("../training data/stance.valid.preprocessed.txt")

(1019, 0.6025515210991168, 0.6025515210991168)

In [129]:
model.test("../training data/stance.valid.preprocessed.txt", k=5)

(1019, 0.2, 1.0)

### Build model for stance/no-stance detection

In [68]:
trainTweetsStanceNoStanceRelCols = trainTweets[["Target","Stance","Tweet"]]

In [69]:
trainTweetsStanceNoStanceRelCols["label"] = np.where(trainTweetsStanceNoStanceRelCols["Stance"] == "NONE", "__label__stance" ,"__label__nostance")

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
trainTweetsWithStanceRelCols["TweetTokenized"] = trainTweetsWithStanceRelCols["Tweet"].apply(createTokens)

In [17]:
trainTweetsStanceNoStanceRelCols["Final"] = trainTweetsStanceNoStanceRelCols["label"] + " " + trainTweetsStanceNoStanceRelCols["Tweet"]

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
trainTweetsStanceNoStanceRelCols[["Final"]].to_csv("../training data/stance.train.preprocessed.txt",
                                                                 header = None, index=False, sep="|",
                                                                quoting = csv.QUOTE_NONE,
                                                               quotechar = '',escapechar='\\')

In [19]:
trainTweetsStanceNoStanceRelCols.to_csv("stance.dataframe.preprocessed.txt",
                                         header= None)

#### test set

In [20]:
testTweetsStanceNoStanceRelCols = testTweets[["Target","Stance","Tweet"]]

In [21]:
testTweetsStanceNoStanceRelCols["label"] = np.where(testTweetsStanceNoStanceRelCols["Stance"] == "NONE",\
                                                    "__label__stance" ,"__label__nostance")

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
#trainTweetsWithStanceRelCols["TweetTokenized"] = trainTweetsWithStanceRelCols["Tweet"].apply(createTokens)

In [23]:
testTweetsStanceNoStanceRelCols["Final"] = testTweetsStanceNoStanceRelCols["label"] + " " + testTweetsStanceNoStanceRelCols["Tweet"]

C:\Users\Mateusz Baryla\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
testTweetsStanceNoStanceRelCols[["Final"]].to_csv("../training data/stance.valid.preprocessed.txt",
                                                                 header = None, index=False, sep="|",
                                                                quoting = csv.QUOTE_NONE,
                                                               quotechar = '',escapechar='\\')

In [25]:
trainTweetsStanceNoStanceRelCols.to_csv("stance.dataframe.preprocessed.txt",
                                         header= None)

### Test model stance no stance 


In [59]:
model = fasttext.train_supervised(input="../training data/stance.train.preprocessed.txt",
                                 lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [60]:
model.get_labels()

['__label__nostance', '__label__stance']

In [61]:
model.save_model("../models/model_stance_nostance.bin")

In [62]:
model.test("../training data/stance.valid.preprocessed.txt")

(1249, 0.8070456365092074, 0.8070456365092074)

In [63]:
model.test_label("../training data/stance.valid.preprocessed.txt")['__label__nostance']

{'precision': 0.8319112627986348,
 'recall': 0.956820412168793,
 'f1score': 0.8900045641259698}

In [64]:
model.test_label("../training data/stance.valid.preprocessed.txt")['__label__stance']

{'precision': 0.42857142857142855,
 'recall': 0.14347826086956522,
 'f1score': 0.21498371335504887}

In [65]:
model.test("../training data/stance.valid.preprocessed.txt", k = 1)[1]

0.8070456365092074

### Hyperparameter tuning

In [24]:
model = fasttext.load_model("../models/model_stance_nostance.bin")

In [80]:
model.test_label("../training data/stance.valid.preprocessed.txt")['__label__nostance']

{'precision': 0.8313458262350937,
 'recall': 0.957801766437684,
 'f1score': 0.8901048791609667}

In [81]:
model.test_label("../training data/stance.valid.preprocessed.txt")['__label__stance']

{'precision': 0.4266666666666667,
 'recall': 0.1391304347826087,
 'f1score': 0.2098360655737705}

In [88]:
res = model.predict("he who exalt himself shall be humbl and he who humbl himself shall be sem", k=2)

In [109]:
model.predict(row["Target"])[0][0]

'__label__nostance'

In [118]:
for i,row in testTweets.iterrows():
    if model.predict(row["Target"])[0][0] == '__label__stance':
        print(i,"stance detected")

In [26]:
model = fasttext.load_model("../models/model_stance_classification.bin")

In [27]:
model.get_labels()

['__label__feminist_movement',
 '__label__hillary_clinton',
 '__label__legalization_of_abortion',
 '__label__atheism',
 '__label__climate_change_is_a_real_concern']

In [28]:
for model_str in model.get_labels():
    print(model_str, model.test_label("../training data/stance_classification/stance.test.preprocessed.tokenized.txt")[model_str])
    print()

__label__feminist_movement {'precision': 0.7282608695652174, 'recall': 0.8340248962655602, 'f1score': 0.7775628626692457}

__label__hillary_clinton {'precision': 0.8712871287128713, 'recall': 0.8110599078341014, 'f1score': 0.8400954653937948}

__label__legalization_of_abortion {'precision': 0.8198757763975155, 'recall': 0.5617021276595745, 'f1score': 0.6666666666666666}

__label__atheism {'precision': 0.7795698924731183, 'recall': 0.7552083333333334, 'f1score': 0.7671957671957672}

__label__climate_change_is_a_real_concern {'precision': 0.5927835051546392, 'recall': 0.8582089552238806, 'f1score': 0.7012195121951219}



In [29]:
model.test("../training data/stance_classification/stance.test.preprocessed.tokenized.txt")

(1019, 0.7546614327772326, 0.7546614327772326)

### Build model for favour/against

In [30]:
trainTweets = pd.read_csv("../training data/training.txt", sep='\t',header=0, encoding = "ISO-8859-1")

In [31]:
testTweets = pd.read_csv("../training data/test.txt", sep='\t',header=0, encoding = "ISO-8859-1")

In [55]:
trainTweets["Target"] = trainTweets["Target"].apply(lambda x:x.replace(" ","_").lower())
testTweets["Target"] = testTweets["Target"].apply(lambda x:x.replace(" ","_").lower())

#### train tweets

In [80]:
trainTweets = trainTweets[trainTweets["Stance"] != "NONE"].reset_index(drop=True)

In [81]:
train_tokenized = pd.read_csv("../training data/stance_classification/stance.train.preprocessed.tokenized.txt", header = None, names=["text"])

In [82]:
train_tokenized["label"] = train_tokenized["text"].apply(lambda x: x.split(" ")[0])
train_tokenized["prep_text"] = train_tokenized["text"].apply(lambda x: " ".join(x.split(" ")[1:]))
train_tokenized[["label","prep_text"]]

,label,prep_text
0,__label__atheism,dear lord thank u for all of ur bless forgiv m...
1,__label__atheism,bless are the peacemak for they shall be call ...
2,__label__atheism,i am not conform to this world i am transform ...
3,__label__atheism,salah should be pray with focus and understand...
4,__label__atheism,and stay in your hous and do not display yours...
5,__label__atheism,if we are unsur whether someth is halal or har...
6,__label__atheism,papa god i pray that you shower me with more p...
7,__label__atheism,now that the s c o c has rule canadian have fr...
8,__label__atheism,everyon is abl to believ in whatev they want f...
9,__label__atheism,the best of you are those who best treat their...


In [83]:
trainTweets["prep_text"] = train_tokenized["prep_text"]

#### test tweets

In [85]:
testTweets = testTweets[testTweets["Stance"] != "NONE"].reset_index(drop=True)

In [86]:
test_tokenized = pd.read_csv("../training data/stance_classification/stance.test.preprocessed.tokenized.txt", header = None, names=["text"])

In [87]:
test_tokenized["label"] = test_tokenized["text"].apply(lambda x: x.split(" ")[0])
test_tokenized["prep_text"] = test_tokenized["text"].apply(lambda x: " ".join(x.split(" ")[1:]))
test_tokenized[["label","prep_text"]]

,label,prep_text
0,__label__atheism,he who exalt himself shall be humbl and he who...
1,__label__atheism,r t prayerbullet i remov nehushtan move of god...
2,__label__atheism,heidtjj benjamin live i have sought the truth ...
3,__label__atheism,god is utter powerless without human intervent...
4,__label__atheism,miracl of multicultur miracl of shadi taqiya t...
5,__label__atheism,this world need a tight group hug tight enough...
6,__label__atheism,moral is not deriv from religion it preced it ...
7,__label__atheism,a god husband know you trust you love you resp...
8,__label__atheism,secular dutchess i be your huckleberri dean mo...
9,__label__atheism,b i b l e big irrelev book of lie and exagger ...


In [88]:
testTweets["prep_text"] = test_tokenized["prep_text"]

#### build model for favour/against

In [95]:
trainTweets["label"] = "__label__"+trainTweets["Stance"].apply(lambda x:x.lower())

In [97]:
trainTweets["Final"] = trainTweets["label"] + " " + trainTweets["prep_text"]

In [104]:
trainTweets[["Final"]].to_csv("../training data/stance_favour_against/stance.train.preprocessed.txt",
                             header = None, index=False, sep="|",
                             quoting = csv.QUOTE_NONE,
                             quotechar = '',escapechar='\\')

#### test set

In [101]:
testTweets["label"] = "__label__"+testTweets["Stance"].apply(lambda x:x.lower())

In [102]:
testTweets["Final"] = testTweets["label"] + " " + testTweets["prep_text"]

In [105]:
testTweets[["Final"]].to_csv("../training data/stance_favour_against/stance.valid.preprocessed.txt",
                             header = None, index=False, sep="|",
                             quoting = csv.QUOTE_NONE,
                             quotechar = '',escapechar='\\')

#### favour/against models

In [106]:
model = fasttext.train_supervised(input="../training data/stance_favour_against/stance.train.preprocessed.txt",
                                 lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [107]:
model.get_labels()

['__label__against', '__label__favor']

In [108]:
model.save_model("../models/model_stance_favour_against.bin")

In [110]:
model.test("../training data/stance_favour_against/stance.valid.preprocessed.txt")

(1019, 0.7311089303238469, 0.7311089303238469)

In [112]:
model.test_label("../training data/stance_favour_against/stance.valid.preprocessed.txt")['__label__against']

{'precision': 0.810126582278481,
 'recall': 0.8055944055944056,
 'f1score': 0.8078541374474053}

In [113]:
model.test_label("../training data/stance_favour_against/stance.valid.preprocessed.txt")['__label__favor']

{'precision': 0.5487012987012987,
 'recall': 0.555921052631579,
 'f1score': 0.5522875816993464}

### Model classification 

In [41]:
model_classification = fasttext.load_model("data/06_models/model_classification.bin")

In [42]:
model_classification.test("data/05_model_input/stanceclassification.test.preprocessed.tokenized.txt")

(812, 0.4027093596059113, 0.4027093596059113)

### Model favor against

In [4]:
model_favour_against = fasttext.load_model("data/06_models/model_favour_against.bin")

In [5]:
model_favour_against.predict("data/05_model_input/stancefavour_against.test.preprocessed.tokenized.txt")

(('__label__none',), array([0.88134712]))

### Model opinion towards 

In [6]:
model_opinion_towards = fasttext.load_model("data/06_models/model_opinion_towards.bin")

In [7]:
model_opinion_towards.test("data/05_model_input/stanceopinion_towards.test.preprocessed.tokenized.txt")

(812, 0.5061576354679803, 0.5061576354679803)

### Model sentiment

In [8]:
model_sentiment = fasttext.load_model("data/06_models/model_sentiment.bin")

In [9]:
model_sentiment.test("data/05_model_input/stancesentiment.test.preprocessed.tokenized.txt")

(812, 0.5788177339901478, 0.5788177339901478)

### Accuracy of trained models

In [22]:
df = pd.read_csv("C:\\Users\\Mateusz\\OneDrive\\Informatyka 6 semestr\\stance-tagger-kedro\\stance-tagger-kedro\\data\\01_raw\\stances.csv")

In [30]:
df['Stance'].value_counts()

AGAINST    2057
FAVOR      1035
NONE        971
Name: Stance, dtype: int64

In [35]:
df_ind_no_stance = df[((df['Opinion towards'] == 'NO ONE') & (df['Stance'] == 'NONE'))].index

In [37]:
df.loc[df_ind_no_stance, 'Target'] = 'None'

In [38]:
df['Target'].value_counts()

Feminist Movement                   911
Hillary Clinton                     898
Legalization of Abortion            838
Atheism                             708
Climate Change is a Real Concern    520
None                                188
Name: Target, dtype: int64

In [40]:
df.to_csv("C:\\Users\\Mateusz\\OneDrive\\Informatyka 6 semestr\\stance-tagger-kedro\\stance-tagger-kedro\\data\\01_raw\\stances_modified.csv", index=False)